<a href="https://colab.research.google.com/github/Rajasri8102/Rajasri-FITA-Capstone-Projects/blob/main/FITA_Final_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Capstone Final Project (“FULL COMPANY PERFORMANCE ANALYSIS”)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# Connect to SQLite DB
conn = sqlite3.connect("fita_capstoneproject_final.db")
cursor = conn.cursor()

# Read SQL file
with open("/content/drive/MyDrive/Colab Notebooks/fita_capstoneproject_final.sql", "r") as f:
    sql_script = f.read()

# Execute schema + inserts
cursor.executescript(sql_script_modified)
conn.commit()

# Load tables into Pandas
customers = pd.read_sql("SELECT * FROM customers", conn)
products  = pd.read_sql("SELECT * FROM products", conn)
sales     = pd.read_sql("SELECT * FROM sales", conn)

# Preview
print(customers.head())
print(products.head())
print(sales.head())

   customer_id customer_name       city    segment signup_date
0            1          Mano    Chennai     Retail  2023-01-10
1            2         Kumar  Bangalore  Corporate  2023-01-15
2            3          Ajay  Hyderabad        SMB  2023-01-20
3            4         Meena      Delhi     Retail  2023-01-25
4            5          Ravi     Mumbai  Corporate  2023-02-01
   product_id  product_name     category  price   cost
0         101        Laptop  Electronics  55000  45000
1         102  Mobile Phone  Electronics  25000  20000
2         103        Tablet  Electronics  30000  25000
3         104    Headphones  Accessories   2000   1200
4         105    Smartwatch  Accessories   8000   5000
   order_id  order_date  customer_id  product_id sales_rep  quantity
0      1001  2023-01-15            1         101     Rep_A         2
1      1002  2023-01-18            2         104     Rep_B         1
2      1003  2023-01-20            3         102     Rep_A         3
3      1004  202

In [ ]:
#📘 2. Data Cleaning
print("📘 2. Data Cleaning")
#✔ Missing Values
customers.isnull().sum()
products.isnull().sum()
sales.isnull().sum()
#✔ Duplicates
customers.duplicated().sum()
products.duplicated().sum()
sales.duplicated().sum()
#✔ Data Types
sales['order_date'] = pd.to_datetime(sales['order_date'])
print(sales.dtypes)
#✔Upper Case
customers["customer_name"] = customers["customer_name"].str.upper()
print(customers.head())


📘 2. Data Cleaning
order_id                int64
order_date     datetime64[ns]
customer_id             int64
product_id              int64
sales_rep              object
quantity                int64
dtype: object
   customer_id customer_name       city    segment signup_date
0            1          MANO    Chennai     Retail  2023-01-10
1            2         KUMAR  Bangalore  Corporate  2023-01-15
2            3          AJAY  Hyderabad        SMB  2023-01-20
3            4         MEENA      Delhi     Retail  2023-01-25
4            5          RAVI     Mumbai  Corporate  2023-02-01


In [ ]:
#📘 STEP 3: DATA TRANSFORMATION (Python)
#➕ Revenue & Profit (Feature Engineering)
merged = sales.merge(products, on='product_id')
merged['Revenue'] = merged['quantity'] * merged['price']
merged['Profit'] = merged['quantity'] * (merged['price'] - merged['cost'])
#📆 Time Features
merged['Year'] = merged['order_date'].dt.year
merged['Month'] = merged['order_date'].dt.month_name()
print(merged.head())


   order_id order_date  customer_id  product_id sales_rep  quantity  \
0      1001 2023-01-15            1         101     Rep_A         2   
1      1002 2023-01-18            2         104     Rep_B         1   
2      1003 2023-01-20            3         102     Rep_A         3   
3      1004 2023-01-25            4         105     Rep_C         1   
4      1005 2023-02-02            5         106     Rep_B         2   

   product_name         category  price   cost  Revenue  Profit  Year  \
0        Laptop      Electronics  55000  45000   110000   20000  2023   
1    Headphones      Accessories   2000   1200     2000     800  2023   
2  Mobile Phone      Electronics  25000  20000    75000   15000  2023   
3    Smartwatch      Accessories   8000   5000     8000    3000  2023   
4       Printer  Office Supplies  12000   9000    24000    6000  2023   

      Month  
0   January  
1   January  
2   January  
3   January  
4  February  


In [ ]:
#📘 STEP 4: DATA MERGING (Python)
final_df = merged.merge(customers, on='customer_id')
print(final_df.head())

   order_id order_date  customer_id  product_id sales_rep  quantity  \
0      1001 2023-01-15            1         101     Rep_A         2   
1      1002 2023-01-18            2         104     Rep_B         1   
2      1003 2023-01-20            3         102     Rep_A         3   
3      1004 2023-01-25            4         105     Rep_C         1   
4      1005 2023-02-02            5         106     Rep_B         2   

   product_name         category  price   cost  Revenue  Profit  Year  \
0        Laptop      Electronics  55000  45000   110000   20000  2023   
1    Headphones      Accessories   2000   1200     2000     800  2023   
2  Mobile Phone      Electronics  25000  20000    75000   15000  2023   
3    Smartwatch      Accessories   8000   5000     8000    3000  2023   
4       Printer  Office Supplies  12000   9000    24000    6000  2023   

      Month customer_name       city    segment signup_date  
0   January          MANO    Chennai     Retail  2023-01-10  
1   Januar

In [ ]:
#📘 STEP 5: KPI CREATION
#Total Revenue
total_revenue = final_df['Revenue'].sum()
print("Total Revenue:", total_revenue)
#Total Profit
total_profit = final_df['Profit'].sum()
print("Total Profit:", total_profit)
#Profit Margin %
profit_margin = (total_profit / total_revenue) * 100
print("Profit Margin %:", profit_margin)
#Avg Order Value
avg_order_value = total_revenue / final_df['order_id'].nunique()
print("Avg Order Value:", avg_order_value)
#Top Product
top_product = final_df.groupby('product_name')['Revenue'].sum().idxmax()
print("Top Product:", top_product)
#Best City
best_city = final_df.groupby('city')['Revenue'].sum().idxmax()
print("Best City:", best_city)
#Best Sales Rep
best_sales_rep = final_df.groupby('sales_rep')['Revenue'].sum().idxmax()
print("Best Sales Rep:", best_sales_rep)
kpis = {
    "Total Revenue": final_df['Revenue'].sum(),
    "Total Profit": final_df['Profit'].sum(),
    "Profit Margin": round(final_df['Profit'].sum() / final_df['Revenue'].sum() * 100, 2),
    "Avg Order Value": round(final_df['Revenue'].sum() / final_df['order_id'].nunique(), 2),
    "Top Product": final_df.groupby('product_name')['Revenue'].sum().idxmax(),
    "Best City": final_df.groupby('city')['Revenue'].sum().idxmax(),
    "Best Sales Rep": final_df.groupby('sales_rep')['Revenue'].sum().idxmax()
}



Total Revenue: 1588500
Total Profit: 372450
Profit Margin %: 23.4466477809254
Avg Order Value: 31770.0
Top Product: Laptop
Best City: Chennai
Best Sales Rep: Rep_B


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#📘 STEP 6: EXPORT DATA (Python → Power BI)
final_df.to_csv("fita_capstoneproject_final.csv", index=False)
